##Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [1]:
#@title update mujoco paths

import os

home = os.path.expanduser('~')
ex_path = '/Documents/Python/BerkeleyDRL_fall2021/hw1'
MJC_PATH = home + '/.mujoco'
if home[:6] == '/Users':
    os.environ['LIBRARY_PATH'] += '/usr/local/lib'
    os.environ['LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)   
else:
    os.environ['LD_LIBRARY_PATH'] = '/usr/local/nvidia/lib64'
    os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)

os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco200'.format(MJC_PATH)
os.environ['MUJOCO_PY_MJKEY_PATH'] = '{}/mjkey.txt'.format(MJC_PATH)

## installation on colab does not find *.so files
## in LD_LIBRARY_PATH, copy over manually instead
## !cp $MJC_PATH/mujoco200/bin/*.so /usr/lib/x86_64-linux-gnu/

Copy over `mjkey.txt` into `~/.mujoco` before this step

In [2]:
#@title clone and install mujoco-py

## cythonize at the first import
import mujoco_py

In [3]:
#@title clone homework repo
'''
%cd $SYM_PATH
!git clone https://github.com/berkeleydeeprlcourse/homework_fall2021.git
%cd homework_fall2021/hw1
%pip install -r requirements_colab.txt
%pip install -e . ''';

In [ ]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start();

In [ ]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
) 

env = wrap_env(gym.make("Ant-v2"))

observation = env.reset()
for i in range(100):
    env.render(mode='rgb_array')
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;
            
env.close()
print('Loading video...')
show_video()

## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2021/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

## Run Behavior Cloning (Problem 1)

In [3]:
#@title imports

import os
import time
import numpy as np

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.bc_agent import BCAgent
from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy

%load_ext autoreload
%autoreload 2

In [4]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  #@markdown expert data
  expert_policy_file = home + ex_path + '/cs285/policies/experts/Ant.pkl' #@param
  expert_data = home + ex_path + '/cs285/expert_data/expert_data_Ant-v2.pkl' #@param
  env_name = 'Ant-v2' #@param ['Ant-v2', 'Humanoid-v2', 'Walker2d-v2', 'HalfCheetah-v2', 'Hopper-v2']
  exp_name = 'test_bc_ant' #@param
  do_dagger = False #@param {type: "boolean"}
  ep_len = 1000 #@param {type: "integer"}
  save_params = False #@param {type: "boolean"}

  num_agent_train_steps_per_iter = 1000 #@param {type: "integer"})
  n_iter = 1 #@param {type: "integer"})

  #@markdown batches & buffers
  batch_size = 1000 #@param {type: "integer"})
  eval_batch_size = 1000 #@param {type: "integer"}
  train_batch_size = 100 #@param {type: "integer"}
  max_replay_buffer_size = 1000000 #@param {type: "integer"}

  #@markdown network
  n_layers = 2 #@param {type: "integer"}
  size = 64 #@param {type: "integer"}
  learning_rate = 5e-3 #@param {type: "number"}

  #@markdown logging
  video_log_freq = 5 #@param {type: "integer"}
  scalar_log_freq = 1 #@param {type: "integer"}

  #@markdown gpu & run-time settings
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

args = Args()


In [5]:
#@title define `BC_Trainer`
class BC_Trainer(object):

    def __init__(self, params):
        #######################
        ## AGENT PARAMS
        #######################

        agent_params = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            'max_replay_buffer_size': params['max_replay_buffer_size'],
            }

        self.params = params
        self.params['agent_class'] = BCAgent ## TODO: look in here and implement this
        self.params['agent_params'] = agent_params

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params) ## TODO: look in here and implement this

        #######################
        ## LOAD EXPERT POLICY
        #######################

        print('Loading expert policy from...', self.params['expert_policy_file'])
        self.loaded_expert_policy = LoadedGaussianPolicy(self.params['expert_policy_file'])
        print('Done restoring expert policy...')

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            n_iter=self.params['n_iter'],
            initial_expertdata=self.params['expert_data'],
            collect_policy=self.rl_trainer.agent.actor,
            eval_policy=self.rl_trainer.agent.actor,
            relabel_with_expert=self.params['do_dagger'],
            expert_policy=self.loaded_expert_policy,
        )


In [6]:
#@title create directory for logging

if args.do_dagger:
    logdir_prefix = 'q2_'  # The autograder uses the prefix `q2_`
    assert args.n_iter>1, ('DAgger needs more than 1 iteration (n_iter>1) of training, to iteratively query the expert and train (after 1st warmstarting from behavior cloning).')
else:
    logdir_prefix = 'q1_'  # The autograder uses the prefix `q1_`
    assert args.n_iter==1, ('Vanilla behavior cloning collects expert data just once (n_iter=1)')

data_path = home + ex_path + '/hw1/data'
if not (os.path.exists(data_path)):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + \
         '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [7]:
## run training
print(args.logdir)
trainer = BC_Trainer(args)
trainer.run_training_loop()

/Users/ikasou/Documents/Python/BerkeleyDRL_fall2021/hw1/hw1/data/q1_test_bc_ant_Ant-v2_18-11-2021_16-40-16
########################
logging outputs to  /Users/ikasou/Documents/Python/BerkeleyDRL_fall2021/hw1/hw1/data/q1_test_bc_ant_Ant-v2_18-11-2021_16-40-16
########################
GPU not detected. Defaulting to CPU.
Loading expert policy from... /Users/ikasou/Documents/Python/BerkeleyDRL_fall2021/hw1/cs285/policies/experts/Ant.pkl
obs (1, 111) (1, 111)
Done restoring expert policy...


********** Iteration 0 ************

Creating offscreen glfw


Training agent using sampled data from replay buffer...
Starting epoch 1
Loss after epoch     1: 0.000
Starting epoch 2
Loss after epoch     2: 0.000
Starting epoch 3
Loss after epoch     3: 0.012
Starting epoch 4
Loss after epoch     4: 0.013
Starting epoch 5
Loss after epoch     5: 0.015
Starting epoch 1
Loss after epoch     1: 0.004
Starting epoch 2
Loss after epoch     2: 0.005
Starting epoch 3
Loss after epoch     3: 0.005
Starting ep

MoviePy - Building file /var/folders/k_/src2x1r50n92465p4dm_73nh0000gn/T/tmpstze4nfs.gif with imageio.


TypeError: iteration over a 0-d array

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

%load_ext tensorboard
%tensorboard --logdir ~/Documents/Python/BerkeleyDRL_fall2021/hw1/data

## Running DAgger (Problem 2)
Modify the settings above:
1. check the `do_dagger` box
2. set `n_iters` to `10`
and then rerun the code.